In [1]:
:set -XTemplateHaskell
:set -XOverloadedStrings
:set -XNoImplicitPrelude
:set -XDeriveFunctor
:set -XDeriveFoldable
:set -XDeriveTraversable
:set -XStandaloneDeriving
:set -XTupleSections
:set -XDeriveGeneric
:set -XDuplicateRecordFields
:set -XMultiParamTypeClasses
:set -XFunctionalDependencies
:set -XFlexibleInstances
:set -XFlexibleContexts


In [2]:
:load TreeUnfold.hs

In [3]:
module Design2HTML where

In [4]:
import Papa
import Data.Tree.Lens
import qualified Data.Sequence as S
import qualified Data.HashMap.Strict as HS
import Data.Tree
import Control.Monad.Zip

import TreeUnfold
import Prelude(showsPrec, showString)


In [22]:
data Coord = Coord {_x :: Int, _y :: Int, _height :: Int, _width :: Int}

data CSSRule = (:=) {_lhs :: String, _rhs :: String}

data DNode = DNode {
                    _did :: String,
                    _title :: String,
                    _coords :: Coord,
                    _content :: Maybe String,
                    _css :: HS.HashMap String [CSSRule],
                    _style :: [CSSRule]}
                    
data Snapshot = Snapshot {
    _src :: String,
    _style :: [CSSRule]}
                    
data DesignDoc = DesignDoc {
    _snapshot :: Snapshot,
    _nodes :: [[DNode]]}


makeFieldsNoPrefix ''Coord
makeFieldsNoPrefix ''DNode
makeFieldsNoPrefix ''Snapshot
makeFieldsNoPrefix ''DesignDoc

deriving instance Show Coord
deriving instance Eq Coord
deriving instance Ord Coord

deriving instance Ord CSSRule
deriving instance Eq CSSRule

instance Show CSSRule where
    showsPrec _ (lhs := rhs) = showString lhs . showString " : " . showString rhs . showString ";"

deriving instance Show DNode

instance Eq DNode where
    a == b = a ^. did == b ^. did

instance Ord DNode where
    a <= b = a ^. coords . y <= b ^. coords . y

deriving instance Show DesignDoc
deriving instance Show Snapshot

In [23]:
import Data.Aeson
import GHC.Generics

deriving instance Generic Coord
deriving instance Generic CSSRule
deriving instance Generic DNode
deriving instance Generic Snapshot
deriving instance Generic DesignDoc

instance FromJSON Coord
instance FromJSON CSSRule
instance FromJSON DNode
instance FromJSON Snapshot
instance FromJSON DesignDoc

instance ToJSON Coord
instance ToJSON CSSRule
instance ToJSON DNode
instance ToJSON Snapshot
instance ToJSON DesignDoc


In [25]:
dummy :: String -> DNode
dummy s = DNode s s (Coord 0 0 0 0) Nothing HS.empty []

wrapper = Node (dummy "header") []

wrap :: String -> HS.HashMap String String -> String ->  String
wrap tag xs x = "<" <> tag <> " " <> (ppMap xs) <> ">" <> x <> "</" <> tag <> ">"

ppMap :: HS.HashMap String String -> String
ppMap = HS.foldlWithKey' (\a k v -> k <> "=" <> "\"" <> v <> "\"" <> " " <> a) ""

ruleToAttr :: [CSSRule] -> String
ruleToAttr = concat . (show <$>)

foldToCSS :: Tree DNode -> String
foldToCSS = foldTree fn
    where
        fn :: DNode -> [String] -> String
        fn (DNode _ _ _ _ css _) strs = HS.foldlWithKey' gn (concat strs) css
        
        gn :: String -> String -> [CSSRule] -> String
        gn a k xs = k <> "{" <> ruleToAttr xs <> "}\n" <> a 

foldToHTML :: Tree DNode -> String
foldToHTML = foldTree fn
    where
        fn :: DNode -> [String] -> String
        fn (DNode i t co c css s) str = wrap "div" attrs (content <> concat str)
            where
                attrs = HS.fromList [
                        ("id",i),
                        ("name", t),
                        ("class", concat $ intersperse " " $ drop 1 <$> HS.keys css),
                        ("style", ruleToAttr s)
                        ]

                        
                content = fromMaybe "" $ wrap "span" HS.empty <$> c


In [26]:
import qualified Data.ByteString.Lazy as BS
import qualified Data.ByteString.Lazy.Char8 as C(pack, putStrLn)

snapshotHTML :: Snapshot -> String
snapshotHTML (Snapshot s xs) = wrap "image" attrs "" 
                        where
                        attrs = HS.fromList [
                            ("style", ruleToAttr xs),
                            ("src", s),
                            ("crossorigin", "anonymous")]
                            
                        
page :: String -> String -> String -> String
page c s b = "<!DOCTYPE html><html><head><meta charset=" <> "utf-8" <> "><style>"
            <> c
            <> "</style></head><body>"
            <> "<div>" 
--             <> s
            <> b
            <> "</div></body></html>"




In [35]:
-- import qualified Data.Text.Encoding as T(decodeUtf8)
import qualified Data.Text as T(pack)
import qualified Data.Text.IO as T(writeFile)
do
    file <- BS.readFile "test.json"
    let f = decode file :: Maybe DesignDoc
        tr = f ^? traverse . nodes . Papa.to (unfoldFullTree wrapper . sort)
        css = foldToCSS <$> tr
        html = foldToHTML <$> tr
        ss = snapshotHTML <$> f ^? traverse . snapshot
--     let h = f ^? traverse . nodes .  Papa.to (foldToHTML . unfoldFullTree wrapper)
--     C.putStrLn $ C.pack $ show (f ^? traverse . nodes)
    T.writeFile "out.html" $ T.pack $ fromMaybe "" $ page <$> css <*> ss <*> html


In [ ]:
:t snapshot